## A notebook to seamlessly take blast output to GO Slim list

This is a notebook meant to run in a working directory. Please set working directory as variable in next cell

In [10]:
cd /Users/sr320/Documents/GitHub/nb-2018/O_lurida

/Users/sr320/Documents/GitHub/nb-2018/O_lurida


In [11]:
wd="analyses/1012"


In [12]:
!mkdir $wd

In [14]:
cd $wd

/Users/sr320/Documents/GitHub/nb-2018/O_lurida/analyses/1012


In [ ]:
!/Applications/bioinfo/ncbi-blast-2.7.1+/bin/blastx \
-query ../../analyses/dml010.tile.1kslop.fa \
-db /Users/sr320/Documents/GitHub/fun-gen/blastdb/uniprot_sprot_r2018_08 \
-out dml010.tile.1kslop-uniprot_blastx4slim.tab \
-evalue 1E-05 \
-num_threads 4 \
-max_target_seqs 10 \
-outfmt 6

### In this directory you will need three files
1) blastout file in format `-6`    
2) Uniprot GO annotation file (340M) available here `http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted`    
3) GOslim file available here `http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted`

In [ ]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/uniprot-SP-GO.sorted

In [ ]:
!curl -O http://owl.fish.washington.edu/halfshell/bu-alanine-wd/17-07-20/GO-GOslim.sorted

In [ ]:
!head dml010.tile.1kslop-uniprot_blastx4slim.tab

In [ ]:
#checking if files in directory
!ls 

### Set blastout file as variable

In [5]:
blastout="Cv_sprot.blastout"


# That should be the last thing you have to Type!

In [6]:
!head -2 $blastout

XM_022430339.1	sp|P00491|PNPH_HUMAN	49.291	282	143	0	116	961	1	282	9.92e-98	310
XM_022430345.1	sp|Q99L04|DHRS1_MOUSE	49.552	335	139	4	213	1196	2	313	2.78e-106	324


In [7]:
#convert pipes to tab
!tr '|' '\t' < $blastout \
> _blast-sep.tab

In [8]:
!head -2 _blast-sep.tab

XM_022430339.1	sp	P00491	PNPH_HUMAN	49.291	282	143	0	116	961	1	282	9.92e-98	310
XM_022430345.1	sp	Q99L04	DHRS1_MOUSE	49.552	335	139	4	213	1196	2	313	2.78e-106	324


In [15]:
#reducing number of columns and sorting to get spid evlaue - 
!awk -v OFS='\t' '{print $3, $1, $13}' < _blast-sep.tab | sort \
> _blast-sort.tab

In [16]:
!head -2 _blast-sort.tab

A0A0B4J1F4	XM_022466820.1	1.07e-21
A0A0B4J1F4	XM_022466821.1	9.79e-22


In [17]:
#joining blast with uniprot annoation file and reducing to three columns UniprotID, Query, All GO terms
!join -t $'\t' \
_blast-sort.tab \
uniprot-SP-GO.sorted \
| cut -f1,2,14 \
> _blast-annot.tab

In [21]:
!head -2 _blast-annot.tab
#should look like
#A1BHN5	CHOYP_AAAS.3.3	GO:0005524; GO:0006298; GO:0030983
#A5PLL7	CHOYP_AAEL_AAEL001208.1.1	GO:0005737; GO:0005783; GO:0005789; GO:0016021; GO:0031625; GO:0061630


A0A0B4J1F4	XM_022466820.1	GO:0005768; GO:0005769; GO:0005886; GO:0051443
A0A0B4J1F4	XM_022466821.1	GO:0005768; GO:0005769; GO:0005886; GO:0051443


## The following is a script modidified from Sam

In [22]:
%%bash 

# This script was originally written to address a specific problem that Rhonda was having



# input_file is the initial, "problem" file
# file is an intermediate file that most of the program works upon
# output_file is the final file produced by the script
input_file="_blast-annot.tab"
file="_intermediate.file"
output_file="_blast-GO-unfolded.tab"

# sed command substitutes the "; " sequence to a tab and writes the new format to a new file.
# This character sequence is how the GO terms are delimited in their field.
sed $'s/; /\t/g' "$input_file" > "$file"

# Identify first field containing a GO term.
# Search file with grep for "GO:" and pipe to awk.
# Awk sets tab as field delimiter (-F'\t'), runs a for loop that looks for "GO:" (~/GO:/), and then prints the field number).
# Awk results are piped to sort, which sorts unique by number (-ug).
# Sort results are piped to head to retrieve the lowest value (i.e. the top of the list; "-n1").
begin_goterms=$(grep "GO:" "$file" | awk -F'\t' '{for (i=1;i<=NF;i++) if($i ~/GO:/) print i}' | sort -ug | head -n1)

# While loop to process each line of the input file.
while read -r line
	do
	
	# Send contents of the current line to awk.
	# Set the field separator as a tab (-F'\t') and print the number of fields in that line.
	# Save the results of the echo/awk pipe (i.e. number of fields) to the variable "max_field".
	max_field=$(echo "$line" | awk -F'\t' '{print NF}')

	# Send contents of current line to cut.
	# Cut fields (i.e. retain those fields) 1-12.
	# Save the results of the echo/cut pipe (i.e. fields 1-12) to the variable "fixed_fields"
	fixed_fields=$(echo "$line" | cut -f1-2)

	# Since not all the lines contain the same number of fields (e.g. may not have GO terms),
	# evaluate the number of fields in each line to determine how to handle current line.

	# If the value in max_field is less than the field number where the GO terms begin,
	# then just print the current line (%s) followed by a newline (\n).
	if (( "$max_field" < "$begin_goterms" ))
		then printf "%s\n" "$line"
			else

			# Send contents of current line (which contains GO terms) to cut.
			# Cut fields (i.e. retain those fields) 13 to whatever the last field is in the curent line.
			# Save the results of the echo/cut pipe (i.e. all the GO terms fields) to the variable "goterms".
			goterms=$(echo "$line" | cut -f"$begin_goterms"-"$max_field")
			
			# Assign values in the variable "goterms" to a new indexed array (called "array"), 
			# with tab delimiter (IFS=$'\t')
			IFS=$'\t' read -r -a array <<<"$goterms"
			
			# Iterate through each element of the array.
			# Print the first 12 fields (i.e. the fields stored in "fixed_fields") followed by a tab (%s\t).
			# Print the current element in the array (i.e. the current GO term) followed by a new line (%s\n).
			for element in "${!array[@]}"	
				do printf "%s\t%s\n" "$fixed_fields" "${array[$element]}"
			done
	fi

# Send the input file into the while loop and send the output to a file named "rhonda_fixed.txt".
done < "$file" > "$output_file"

In [23]:
!head _blast-GO-unfolded.tab
#should look like
#A1BHN5	CHOYP_AAAS.3.3	GO:0005524
#A1BHN5	CHOYP_AAAS.3.3	GO:0006298
#A1BHN5	CHOYP_AAAS.3.3	GO:0030983
#A5PLL7	CHOYP_AAEL_AAEL001208.1.1	GO:0005737
#A5PLL7	CHOYP_AAEL_AAEL001208.1.1	GO:0005783


A0A0B4J1F4	XM_022466820.1	GO:0005768
A0A0B4J1F4	XM_022466820.1	GO:0005769
A0A0B4J1F4	XM_022466820.1	GO:0005886
A0A0B4J1F4	XM_022466820.1	GO:0051443
A0A0B4J1F4	XM_022466821.1	GO:0005768
A0A0B4J1F4	XM_022466821.1	GO:0005769
A0A0B4J1F4	XM_022466821.1	GO:0005886
A0A0B4J1F4	XM_022466821.1	GO:0051443
A0A0B4J1F4	XM_022466822.1	GO:0005768
A0A0B4J1F4	XM_022466822.1	GO:0005769


In [166]:
!awk '{print $3,"\t" $2}' _blast-GO-unfolded.tab | gsort -V > _blast-GO-unfolded.sorted

In [277]:
!head _blast-GO-unfolded.sorted 
#extra space was removed tw

GO:0000002	XM_022432754.1
GO:0000002	XM_022434736.1
GO:0000002	XM_022434853.1
GO:0000002	XM_022434856.1
GO:0000002	XM_022435703.1
GO:0000002	XM_022437274.1
GO:0000002	XM_022437275.1
GO:0000002	XM_022437276.1
GO:0000002	XM_022441113.1
GO:0000002	XM_022442505.1


In [204]:
!head GO-GOslim.sorted

GO:0000001	mitochondrion inheritance	cell organization and biogenesis	P
GO:0000002	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000003	reproduction	other biological processes	P
GO:0000006	high affinity zinc uptake transmembrane transporter activity	transporter activity	F
GO:0000007	low-affinity zinc ion transmembrane transporter activity	transporter activity	F
GO:0000009	"alpha-1,6-mannosyltransferase activity"	other molecular function	F
GO:0000010	trans-hexaprenyltranstransferase activity	other molecular function	F
GO:0000011	vacuole inheritance	cell organization and biogenesis	P
GO:0000012	single strand break repair	DNA metabolism	P
GO:0000012	single strand break repair	stress response	P


In [278]:
!join -1 1 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted | head

GO:0000002	XM_022432754.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	XM_022434736.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	XM_022434853.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	XM_022434856.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	XM_022435703.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	XM_022437274.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	XM_022437275.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	XM_022437276.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	XM_022441113.1	mitochondrial genome maintenance	cell organization and biogenesis	P
GO:0000002	XM_022442505.1	mitochondrial genome maintenance	cell organization and biogenesis	P
join: stdout: Broken pipe


In [279]:
#joining files to get GOslim for each query (with duplicate GOslim / query removed)
!join -1 1 -2 1 -t $'\t' \
_blast-GO-unfolded.sorted \
GO-GOslim.sorted \
| uniq | awk -F'\t' -v OFS='\t' '{print $2, $4, $5}' \
> Blastquery-GOslim.tab
!wc -l Blastquery-GOslim.tab

  435993 Blastquery-GOslim.tab


In [280]:
!head Blastquery-GOslim.tab
!wc -l Blastquery-GOslim.tab

XM_022432754.1	cell organization and biogenesis	P
XM_022434736.1	cell organization and biogenesis	P
XM_022434853.1	cell organization and biogenesis	P
XM_022434856.1	cell organization and biogenesis	P
XM_022435703.1	cell organization and biogenesis	P
XM_022437274.1	cell organization and biogenesis	P
XM_022437275.1	cell organization and biogenesis	P
XM_022437276.1	cell organization and biogenesis	P
XM_022441113.1	cell organization and biogenesis	P
XM_022442505.1	cell organization and biogenesis	P
  435993 Blastquery-GOslim.tab


# Get P - query and slim

In [284]:
!awk -F"\t" '$3 == "P" { print $1"\t"$2 }' Blastquery-GOslim.tab | sort > Blastquery-GOslim.sorted
!head Blastquery-GOslim.sorted


XM_022430339.1	cell cycle and proliferation
XM_022430339.1	developmental processes
XM_022430339.1	other biological processes
XM_022430339.1	other biological processes
XM_022430339.1	other metabolic processes
XM_022430339.1	other metabolic processes
XM_022430339.1	other metabolic processes
XM_022430339.1	other metabolic processes
XM_022430339.1	other metabolic processes
XM_022430339.1	other metabolic processes
